<a href="https://colab.research.google.com/github/Bio2Byte/public_notebooks/blob/main/constava_examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Constava - Examples for common use cases

***
***

In this Jupyter notebook common use cases of the Constava software are described. Code examples are given for the Python interface, as well as their corresponding command line commands.


## Table of contents

***

1. [Preparations](#preparations)
2. [Basic usage](#basic-usage)
    1. [Extracting backbone dihedrals from a conformationa ensemble](#extracting-backbone-dihedrals)
    2. [Analyzing a conformational ensemble](#analyzing-a-conformational-ensemble)
    3. [Generating and loading conformational state models](#generating-and-loading-csm)
3. [Visualizing the results](#visualization)
    1. [Plotting the results](#visualization-plotting)
    2. [Mapping a property on the protein structure](##visualization-structure)
4. [Specialized use cases](#specialized-use-cases)
    1. [Using gmx chi output](#using-gmx-chi-output)
    1. [Custom confromational state models](#custom-csm)
    1. [Speeding up the analysis](#speeding-up-the-analysis)


<a name="preparations"></a>

## 1 Preparations

***

Execute this cell to get the example data and install necessary packages.

In [ ]:
%%capture
# Install constava and necessary dependencies
!pip install constava matplotlib py3dmol secstructartist
# Download and unzip test data
!wget -O constava_examples_data.zip https://zenodo.org/record/8410767/files/constava_examples_data.zip?download=1
!unzip -n constava_examples_data.zip

<a name="basic-usage"></a>

## 2 Basic usage

***

The software provides two modes of interaction. Shell user may use the software from the command line, while users skilled in Python can import it as a module.
In the following section the three basic functionalities of the software are described.

<a name="extracting-backbone-dihedrals"></a>

### 2.1 Extracting backbone dihedrals from a conformationa ensemble

To analyze a conformational ensemble, the user needs to provide the backbone dihedral angles $(\phi, \psi)$ extracted from the ensemble.

Below an exemplary code for the extraction of $\phi$- and $\psi$-angles from an MD trajectory is given. The `calculate_dihedrals` function returns a `pandas.DataFrame` object with the backbone dihedral information along the trajectory. This information is then stored in a csv file.

A warning will inform you, that no dihedral angles are extracted for the first and last residue in a protein. This is expected as for terminal residues only $\phi$ or $\psi$ is defined.


In [ ]:
import pandas as pd
from constava.utils.dihedrals import calculate_dihedrals

# Calculate dihedrals as a DataFrame
dihedrals = calculate_dihedrals(structure="./1v66.pdb", trajectory="./1v66.xtc", degree=True)

# Write dihedrals out as a csv
dihedrals.to_csv("extracted_dihedrals.csv", index=False, float_format="%.1f")
dihedrals.head()

The same behavior can be achieved through the command line:

```sh
constava dihedrals -O \
    -s "1v66.pdb" \
    -f "1v66.xtc" \
    -o "extracted_dihedrals.csv" \
    --degrees \
    --precision 1
```


<a name="analyzing-a-conformational-ensemble"></a>

### 2.2 Analyzing a conformational ensemble

After the backbone dihedral angles for a conformational ensemble have been
calculated, conformational state propensities for each residue can be inferred.

The `Constava`-class provides an easy to use interface. Parameters can be
altered any time using the `set_param()` method. Conformational state models
that are initialized are stored inside the `Constava`-instance. That way, by
changing the `input_files` and `output_file` attributes, the  calculation can
be run on multiple ensembles without refitting the conformational state model.

In [ ]:
from constava import Constava

# Define input and output files
PDBID = "1v66"
input_file = f"./{PDBID}_dihedrals.csv"
output_file = "./constava_results.csv"

# Initialize Constava Python interface with parameters
c = Constava(
    input_files = input_file,
    output_file = output_file,
    input_degrees=True,
    window = None,
    bootstrap = [3,5,10,25],
    verbose = 1)

# Alter parameters after initialization
c.set_param("window", [1,3,5])
print(c.show_params())

# Run the calculation and write results
c.run()

An equivalent command from the command line would be:

```sh
# Run constava with debug-level output
constava analyze \
    -i "1v66_dihedrals.csv" \
    -o "constava_results.csv" \
    --window 1 3 5 \
    --bootstrap 3 5 10 25 \
    --degrees \
    -v
```

Note, that in the command line each call fits their own conformational state model. To use the same model in multiple runs, first use the `constava fit-model` subcommand to save a model, and then load the model using the `--load-model` flag (see [below](#generating-and-loading-csm)).

<a name="generating-and-loading-csm"></a>

### 2.3 Generating and loading conformational state models

Conformational state models are usually fitted at runtime. This generates a small but unnecessary computational overhead when running multiple analyses using the same model.

To aleviate this issue, a model can be pre-trained and stored to be used in subsequent analyses directly.

**Note:** Storing conformational state models is intended for quickly running multiple analyses using the same model. They are **not intended for long-term storage or sharing** your conformational state models.
To store or share custom conformational state models, provide the training data as well as the model-fitting parameters.

In [ ]:
import glob
from constava import Constava

# Fit a KDE-model on default dataset with bandwidth==0.13
c = Constava(verbose = 2)
csmodel = c.fit_csmodel(kde_bandwidth = .13)

# Write the fitted model out as a pickle
csmodel.dump_pickle("stored_model.pkl")

# Use the new model to analyze a confromational ensemble
PDBID = "1v66"
input_file = f"./{PDBID}_dihedrals.csv"
output_file = "./constava_results.csv"

c = Constava(
    input_files = input_file,
    output_file = output_file,
    model_load = "stored_model.pkl",
    input_degrees=True,
    window = [1, 5, 10, 25],
    verbose = 2)
c.run()

<a name="generating-and-loading-csm-shell"></a>

To fit, save and load a model from the command line use the following commands:

```sh
# Fit the model on default dataset
constava fit-model -v \
    -o "stored_model.pkl" \
    --kde-bandwidth 0.13

# Run a prediction using the saved model
constava analyze \
    -i "1v66_dihedrals.csv" \
    -o "1v66_constava.csv" \
    --load-model "stored_model.pkl" \
    --window 1 5 10 25 \
    --degrees \
    -vv
```

<a name="visualization"></a>

## 3 Visualizing the results


<a name="visualization-plotting"></a>

### 3.1 Plotting the results

Below constava results are plotted for a simple protein structure with four helices.

In [ ]:
import matplotlib.pyplot as plt
import secstructartist as ssa
import numpy as np
import pandas as pd

# Load the results csv
states = ["coreSheet", "surrSheet", "Other", "Turn", "surrHelix", "coreHelix"]
results = pd.read_csv("1v66_results.csv")

# Select all values, where method was 'window/5/'
win5 = results.loc[results["#Method"] == "window/5/"]
resids = win5["ResIndex"].to_numpy()
vari = win5["Variability"].to_numpy()
prop = win5[states].to_numpy()
secstruct = "LLHHHHHHHHHHLLHHHHHHHHHHHLLLLLLLHHHHHHHHHHHHHHLLLHHHHHHHHHHHHHL"

# Generate a plot
fig, (ax0, ax1, ax2) = plt.subplots(3,1, figsize=(4, 2.5), dpi=120, sharex=True,
                                    gridspec_kw={"hspace": 0., "height_ratios": [.7,2,3]})

# Plot secondary structure
ssa.draw(secstruct, resids, ax=ax0)
ax0.set_ylim([-.7, .55])
ax0.set_axis_off()

# Plot conformational state variability
ax1.grid(True, "major", "y")
ax1.plot(resids, vari)
ax1.set_ylabel("Variablility")
ax1.set_yticks(np.arange(0, .8, .2))

# Plot conformational state propensities as heatmap
ax2.imshow(prop.T, vmin=0, vmax=1, cmap="Blues", aspect="auto",
           extent=(np.min(resids)-.5, np.max(resids)+.5, len(states) -.5, -.5))
ax2.set_yticks(np.arange(len(states)))
ax2.set_yticklabels(states)
ax2.set_xlabel("Residue index")

plt.show()

<a name="visualization-structure"></a>

### 3.2 Mapping a property on the protein structure

Analogous to plotting the data, the constava results can also be mapped on a protein structure, using the B-factor column of the PDB file format. Since most values calculated by constava are in the range `[0, 1], it is useful to scale these values up before writing them to a PDB file to use the whole range of the B-factor column.

In [ ]:
import numpy as np
from matplotlib import cm
import py3Dmol

def constava2pdb(input_pdb, output_pdb, df, state_label, factor = 1):
    """Write values scaled by `factor` to temp factor column in PDB"""
    with open(input_pdb, "r") as ipdb, open(output_pdb, "w") as opdb:
        for ln in ipdb:
            try:
                assert ln.startswith("ATOM  ")
                resn, resi = ln[17:20], int(ln[22:26])
                bfac = df.loc[(df["ResName"] == resn) & (df["ResIndex"] == resi), [state_label]].iloc[0,0]
            except (AssertionError, IndexError):
                opdb.write(ln)
            else:
                opdb.write("{0}{2:6.2f}{1}".format(ln[:60], ln[66:], factor*bfac))

# Try out other properties: coreHelix, surrHelix, coreSheet, surrSheet, Turn, Other, Variability
PROPERTY = "Turn"

# Write "Conformational State Variability" values to the pdb file
results = pd.read_csv("1v66_results.csv")
bstrp5 = results.loc[results["#Method"] == "bootstrap/5/500//"]
constava2pdb("1v66.pdb", "1v66_bfactor.pdb", bstrp5, "coreHelix", factor=100)

# Make a custom linear gradient
mygradient = []
for x in np.linspace(.1, .9, 50):
    r,g,b,_ = cm.viridis(x)
    mygradient.append(f"#{int(255*r):x}{int(255*g):x}{int(255*b):x}")

# Display protein colored by factor
with open("1v66_bfactor.pdb", "r") as fhandle:
    struct = fhandle.read()

view = py3Dmol.view(width=500, height=500)
view.setBackgroundColor('black')
view.setViewStyle({'style':'outline','color':'#888','width':0.1})
view.addModel(struct, "pdb")
view.setStyle({"model": 0}, {'cartoon': {'colorscheme': {
    'prop':'b', 'gradient': "linear", "colors": mygradient, 'min':0,'max':70}}})
view.addSurface(py3Dmol.VDW, {"opacity": .5, 'colorscheme':{
    'prop':'b', 'gradient':"linear", "colors": mygradient, 'min':0, 'max':70}}, {"model": 0})
view.zoomTo()
view.show()


<a name="specialized-use-cases"></a>

## 4 Specialized use cases

***

In the following section we will discuss special use cases, that likely only apply to a small number of users.

<a name="using-gmx-chi-output"></a>

### 4.1 Using `gmx chi` output

Users of the [GROMACS](https://manual.gromacs.org/current/index.html) simulation package may use GROMACS' `gmx chi` module to extract the backbone dihedral angles:

```sh
gmx chi -s <struct.gro> -f <trajectory.xtc> -rama
```

This creates a couple of files named `ramaPhiPsi<RESN><RESID>.xvg` with `<RESN>`being the residue's code and `<RESID>` being the residues index. Constava can read those files directly.


In [ ]:
import glob
from constava import Constava

# Define input and output files
PDBID = "1v66"
input_files = glob.glob(f"./gmxchi/ramaPhiPsi*.xvg")
output_file = "./constava_results.csv"

# Initialize Constava Python interface with parameters
c = Constava(
    input_files = input_files,
    output_file = output_file,
    input_degrees=True,
    window = None,
    bootstrap = [3,5,10,25],
    verbose = 1)

# Alter parameters after initialization
c.set_param("window", [1,3,5])
print(c.show_params())

# Run the calculation and write results
c.run()

[2023-10-05 14:52:19] Constava: Initializing python interface...


INFO:Constava:Constava: Initializing python interface...


[2023-10-05 14:52:19] Setting `window = [1, 3, 5]`


INFO:Constava:Setting `window = [1, 3, 5]`


ConstavaParameters(input_files=['./gmxchi/ramaPhiPsiLEU23.xvg', './gmxchi/ramaPhiPsiSER50.xvg', './gmxchi/ramaPhiPsiLYS30.xvg', './gmxchi/ramaPhiPsiGLN54.xvg', './gmxchi/ramaPhiPsiARG33.xvg', './gmxchi/ramaPhiPsiTYR25.xvg', './gmxchi/ramaPhiPsiHIS43.xvg', './gmxchi/ramaPhiPsiLEU45.xvg', './gmxchi/ramaPhiPsiGLU6.xvg', './gmxchi/ramaPhiPsiLYS34.xvg', './gmxchi/ramaPhiPsiPRO51.xvg', './gmxchi/ramaPhiPsiALA41.xvg', './gmxchi/ramaPhiPsiARG15.xvg', './gmxchi/ramaPhiPsiVAL11.xvg', './gmxchi/ramaPhiPsiLYS56.xvg', './gmxchi/ramaPhiPsiSER13.xvg', './gmxchi/ramaPhiPsiGLY48.xvg', './gmxchi/ramaPhiPsiALA26.xvg', './gmxchi/ramaPhiPsiLYS40.xvg', './gmxchi/ramaPhiPsiMET10.xvg', './gmxchi/ramaPhiPsiARG63.xvg', './gmxchi/ramaPhiPsiGLU59.xvg', './gmxchi/ramaPhiPsiGLN20.xvg', './gmxchi/ramaPhiPsiMET1.xvg', './gmxchi/ramaPhiPsiGLY24.xvg', './gmxchi/ramaPhiPsiLEU14.xvg', './gmxchi/ramaPhiPsiILE57.xvg', './gmxchi/ramaPhiPsiTHR39.xvg', './gmxchi/ramaPhiPsiHIS35.xvg', './gmxchi/ramaPhiPsiVAL21.xvg', './gmxchi/

...

INFO:Constava:Reading dihedrals from 65 files...


[2023-10-05 14:52:20] Starting inference...


INFO:Constava:Starting inference...
100%|██████████| 65/65 [01:52<00:00,  1.73s/residues]

[2023-10-05 14:54:13] Writing results to file: ./constava_results.csv



INFO:Constava:Writing results to file: ./constava_results.csv


An equivalent command from the command line would be:

```sh
# Run constava
PDBID="1v66"
constava analyze \
    -i ${PDBID}/ramaPhiPsi*.xvg \
    -o constava_results.csv \
    --window 1 3 5 \
    --bootstrap 3 5 10 25 \
    --degrees \
    -v
```

<a name="custom-csm"></a>

### 4.2 Custom confromational state models

Besides the default conformational state model from the associated publication,
users can also define their own conformational states. This is done by providing
a JSON file with $(\phi, \psi)$ dihedral pairs that define each conformational state. An example would look like this:

> ```python
> {
>     "MyHelixState": [
>         [phi1, psi1],
>         [phi2, psi2],
>         ... ],
>     "MyOtherState": [
>         [phi1, psi1],
>         [phi2, psi2],
>         ... ],
>     ...
> }
> ```

Here, `phi1`, `psi1`, etc. are floats representing the dihedral in radians or degrees (use of --degrees flag required, see below).

How to fit and use the probabilistic conformational state model from such a
JSON file is described in the code block below.

In [ ]:
from constava import Constava

# Fit the grid-interpolation model
c = Constava(verbose = 1)
csmodel = c.fit_csmodel(model_type = "kde",
                        model_data = "custom_confstates.json",
                        model_data_degrees = True,
                        kde_bandwidth = .13)

# Write the fitted model out as a pickle
csmodel.dump_pickle("custom_model.pkl")

# Use the new model to analyze a confromational ensemble
PDBID = "1v66"
input_files = glob.glob(f"./{PDBID}_dihedrals.csv")
output_file = "constava_results.csv"
c = Constava(
    input_files = input_files,
    output_file = output_file,
    model_load = "custom_model.pkl",
    input_degrees=True,
    window = [1, 5, 10, 25],
    verbose = 1)
c.run()

[2023-10-05 14:54:13] Constava: Initializing python interface...


INFO:Constava:Constava: Initializing python interface...


[2023-10-05 14:54:13] Fitting model to data in: custom_confstates.json


INFO:Constava:Fitting model to data in: custom_confstates.json


[2023-10-05 14:54:13] ... model fitted: <kdeModel states=('helix', 'sheet', 'coil', 'N/A')>


...

INFO:Constava:Reading dihedrals from 1 files...


[2023-10-05 14:54:13] Starting inference...


INFO:Constava:Starting inference...
100%|██████████| 63/63 [00:17<00:00,  3.61residues/s]

[2023-10-05 14:54:30] Writing results to file: constava_results.csv



INFO:Constava:Writing results to file: constava_results.csv


The same actions can be taken from the command line running:

```sh
# Fit the model on custom dataset
constava fit-model -v \
    -i "custom_confstates.json" \
    -o "custom_model.pkl" \
    --kde-bandwidth 0.13 \
    --degrees

# Run a prediction using the saved model
constava analyze -v \
    -i "1v66_dihedrals.csv" \
    -o "1v66_constava.csv" \
    --load-model "custom_model.pkl" \
    --window 1 5 10 25 \
    --degrees
```

<a name="speeding-up-the-analysis"></a>

### 4.3 Speeding up the analysis

Analyzing large conformational ensembles with constava can be time consuming. The bottleneck here is the inference of propensities for each conformational state from their respective probability density functions represented by Gaussian kernel density estimators (KDEs).

To speed up calculations, users can use the `grid` model type. This replaces the KDEs by a grid representing fixed points on the probability density function. The propensities are consequently inferred using linear interpolation between the closest grid points.

While this is less accurate than direct inference from the KDEs, using the `grid` model type has the potential to speed up calculations significantly. Note, that fitting a grid model may take some time since the grid points need first to be inferred from a KDE model.


In [ ]:
from constava import Constava

# Fit the grid-interpolation model
c = Constava(verbose = 1)
csmodel = c.fit_csmodel(model_type = "grid",
                        kde_bandwidth = .13,
                        grid_points = 6400)

# Write the fitted model out as a pickle
csmodel.dump_pickle("grid_model.pkl")

# Use the new model to analyze a confromational ensemble
PDBID = "1v66"
input_files = glob.glob(f"./{PDBID}_dihedrals.csv")
output_file = "constava_results.csv"
c = Constava(
    input_files = input_files,
    output_file = output_file,
    model_load = "grid_model.pkl",
    input_degrees=True,
    window = [1, 5, 10, 25],
    verbose = 1)
c.run()

To fit, save and load a grid-interpolation model from the command line use the
following commands:

```sh
# Fit the grid-interpolation model from default dataset
constava fit-model -v \
    -o grid_model.pkl \
    --model-type grid \
    --kde-bandwidth 0.13 \
    --grid-points 6400

# Run a prediction using the saved model
constava analyze -v \
    -i "1v66_dihedrals.csv" \
    -o "constava_results.csv" \
    --load-model grid_model.pkl \
    --window 1 5 10 25 \
    --degrees
```